In [1]:
import os
from tifffile import imread, imwrite
import numpy as np
from scipy.ndimage import shift

In [2]:
# === PARAMETERS ===
INPUT_FOLDER = r"L:\43-RVZ\AIMicroscopy\Mitarbeiter\2_Data\1_NikonTi2\2025_07_29 60xWI 1.5x nanobody titration\TIF\stacked"
OUTPUT_FOLDER = os.path.join(INPUT_FOLDER, 'aligned_stacks')
os.makedirs(OUTPUT_FOLDER, exist_ok=True)
SHIFT_X = -5         # Pixels to shift in X direction
# A positive value shifts the fluorescence channel to the right
# A negative value shifts the fluorescence channel to the left
SHIFT_Y = -14       # Pixels to shift in Y direction
# A positive value shifts the fluorescence channel downwards.
# A negative value shifts the fluorescence channel upwards.

CROP_SIZE_X = 3000     # Final X dimension (width)
CROP_SIZE_Y = 3000     # Final Y dimension (height)

In [3]:
def align_and_crop_tif_stack(
    input_folder,
    shift_x,
    shift_y,
    crop_size_x,
    crop_size_y
):
    # Make sure input_folder is an absolute path
    input_folder = os.path.abspath(input_folder)
    # Output folder is always input_folder/aligned_stacks
    output_folder = os.path.join(input_folder, 'aligned_stacks')
    os.makedirs(output_folder, exist_ok=True)
    
    # Get TIFF files and print count
    tif_files = [f for f in os.listdir(input_folder) 
                 if f.lower().endswith(('.tif', '.tiff')) and os.path.isfile(os.path.join(input_folder, f))]
    print(f"Found {len(tif_files)} TIFF files in '{input_folder}'")
    print(f"Saving aligned images to '{output_folder}'")

    # Process each TIFF file
    for i, filename in enumerate(tif_files, 1):
        img_path = os.path.join(input_folder, filename)
        img = imread(img_path)
        
        # Shift fluorescence channel (channel 1)
        shifted_fluo = shift(img[1, ...], (shift_y, shift_x), mode='nearest')
        aligned_img = np.stack([img[0, ...], shifted_fluo], axis=0)
        
        # Calculate crop boundaries
        h, w = aligned_img.shape[1:]  # Height and width are the 2nd and 3rd dimensions
        start_y = (h - crop_size_y) // 2
        start_x = (w - crop_size_x) // 2
        cropped = aligned_img[:,
            start_y:start_y + crop_size_y,
            start_x:start_x + crop_size_x
        ]
        
        # Save processed image
        output_path = os.path.join(output_folder, filename)
        imwrite(output_path, cropped)
        print(f"[{i}/{len(tif_files)}] Processed: {filename} => {output_path}")

In [4]:
# === RUN SCRIPT ===
align_and_crop_tif_stack(
    input_folder=INPUT_FOLDER,
    shift_x=SHIFT_X,
    shift_y=SHIFT_Y,
    crop_size_x=CROP_SIZE_X,
    crop_size_y=CROP_SIZE_Y
)

Found 93 TIFF files in 'L:\43-RVZ\AIMicroscopy\Mitarbeiter\2_Data\1_NikonTi2\2025_07_29 60xWI 1.5x nanobody titration\TIF\stacked'
Saving aligned images to 'L:\43-RVZ\AIMicroscopy\Mitarbeiter\2_Data\1_NikonTi2\2025_07_29 60xWI 1.5x nanobody titration\TIF\stacked\aligned_stacks'
[1/93] Processed: MreB 100nM DIC_25_500 Cy5_20_200ms001_stack.tif => L:\43-RVZ\AIMicroscopy\Mitarbeiter\2_Data\1_NikonTi2\2025_07_29 60xWI 1.5x nanobody titration\TIF\stacked\aligned_stacks\MreB 100nM DIC_25_500 Cy5_20_200ms001_stack.tif
[2/93] Processed: MreB 100nM DIC_25_500 Cy5_50_200ms001_stack.tif => L:\43-RVZ\AIMicroscopy\Mitarbeiter\2_Data\1_NikonTi2\2025_07_29 60xWI 1.5x nanobody titration\TIF\stacked\aligned_stacks\MreB 100nM DIC_25_500 Cy5_50_200ms001_stack.tif
[3/93] Processed: MreB 100nM DIC_25_500 Cy5_50_200ms002_stack.tif => L:\43-RVZ\AIMicroscopy\Mitarbeiter\2_Data\1_NikonTi2\2025_07_29 60xWI 1.5x nanobody titration\TIF\stacked\aligned_stacks\MreB 100nM DIC_25_500 Cy5_50_200ms002_stack.tif
[4/93] 